In [ ]:
##!/usr/bin/env python
"""plot_advection.py

Script plots contour maps of the heat and moisture advection
d02: 800m resolution with urban LCZs

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Tue Sep  8 09:10:48 AEST 2020

"""

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Load Packages

In [1]:
#from __future__ import division
import numpy as np
import pandas as pd
import math
import netCDF4 as nc
import sys
import os
import glob as glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
from matplotlib import cm
from matplotlib.collections import LineCollection
import common_functions as cf
import datetime as dt
import wrf
from scipy import stats
import metpy.calc as mpcalc


### Experimental Details of the Model Data

In [2]:
# Simulation Period
syear = 2017
smon = 1
sday = 2 
eyear = 2017
emon = 2
eday = 28  # Add an extra day so that the 27th Feb data is included
simlen = dt.datetime(eyear,emon,eday) - dt.datetime(syear,smon,sday)
nst = (simlen.days * 24 * 6) # No. simulations days x 24 hours in a day x 6 history intervals per hour

# Dates - Used for subsetting the AWS data so you pick the day before the start date and the day after the end date
sdate = "2017-01-01"
edate = "2017-02-28"

# Data directory 
datadir='/g/data/w97/azh561/WRF/'
ensmem = ['sydney800m','sydney800m_06H','sydney800m_12H','sydney800m_18H','sydney800m_00H'] 
domain = ["d02","d02","d02","d02","d02"]
nmem = len(ensmem)

# Landsea mask
mask_file='/g/data/w97/azh561/WRF/sydney800m/geo_em.%s.nc' %(domain[0])
f = nc.Dataset(mask_file)
lu = f.variables['LU_INDEX'][0,:,:]
luf = f.variables['LANDUSEF'][0,:,:,:]
lat2d = f.variables['XLAT_M'][0,:,:]
lontmp = f.variables['XLONG_M'][0,:,:]
lon2d = np.where(lontmp<0.0,lontmp+360,lontmp)
hgt2d = f.variables['HGT_M'][0,:,:]
lsmask = f.variables['LANDMASK'][0,:,:]
clon = f.getncattr('CEN_LON')
nlu = f.getncattr('NUM_LAND_CAT')
iswater = f.getncattr('ISWATER')
nlat,nlon = lon2d.shape
f.close()


lat1d = lat2d[:,0]
lon1d = lon2d[0,:]
# Use helper function defined above to calculate distance
# between lat/lon grid points
dx, dy = mpcalc.lat_lon_grid_deltas(lon1d, lat1d)
# Because of the way the data are returned we need a negative spacing. This
# will be easier in the next version of MetPy.
dy *= -1

nlev = 44

# LCZs
LCZnm = ['Compact high-rise','Compact midrise','Compact low-rise','Open high-rise',
         'Open low-rise','Lightweight low-rise','Large low-rise','Sparsely built','Heavy industry']

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())
fig_name_prefix='LCZ_'
if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)


In [3]:
start = dt.datetime(syear,smon,sday,0,0,0)
end = dt.datetime(eyear,emon,eday,0,0,0)
days = (end - start).days
ntim = days * 24 * 60
datelist = [start + dt.timedelta(minutes=x) for x in range(ntim+1)]
# Get the day-month hour-minutes on 10 minute interval
ftimes = np.asarray([datelist[x].strftime("%m-%d %H-%M") for x in range(ntim+1)])[::10]
fhours = np.asarray([datelist[x].strftime("%H") for x in range(ntim+1)])[::10]

### Split analysis by heatwave periods

From the Bureau of Meteorology Special Climate Statement 61 there were 3 heatwaves:

    10-14 January
    17-21 January
    31 January - 12 February 
    
For the latter heatwave this was terminated by a cold front.

So here we examine separately the week before the first heatwave, each heatwave period and the week after the third heatwave

In [4]:
PRES = [i for i in range(len(ftimes)) if ftimes[i] in ['01-03 00-00']][0]
PREF = [i for i in range(len(ftimes)) if ftimes[i] in ['01-10 00-00']][0]

HW1S = [i for i in range(len(ftimes)) if ftimes[i] in ['01-10 00-00']][0]
HW1F = [i for i in range(len(ftimes)) if ftimes[i] in ['01-15 00-00']][0]

HW2S = [i for i in range(len(ftimes)) if ftimes[i] in ['01-17 00-00']][0]
HW2F = [i for i in range(len(ftimes)) if ftimes[i] in ['01-22 00-00']][0]

HW3S = [i for i in range(len(ftimes)) if ftimes[i] in ['01-31 00-00']][0]
HW3F = [i for i in range(len(ftimes)) if ftimes[i] in ['02-13 00-00']][0]

POSTS = [i for i in range(len(ftimes)) if ftimes[i] in ['02-13 00-00']][0]
POSTF = [i for i in range(len(ftimes)) if ftimes[i] in ['02-20 00-00']][0]


In [5]:
filelist = sorted(glob.glob('%s/%s/WRF_output/%s/wrfout_%s_2017-*' %(datadir,ensmem[1],domain[1],domain[1])))

In [8]:
filelist[0][65:75]

'2017-01-02'

### Extract model data corresponding to an East-West Transect through the city (latitude index of 174)

- run this on gadi to avoid memory limits

In [ ]:
for mm in range(nmem):
    
    # Files list
    filelist = sorted(glob.glob('%s/%s/WRF_output/%s/wrfout_%s_2017-*' %(datadir,ensmem[mm],domain[mm],domain[mm])))
    nfile = len(filelist)
    for ff in range(int(nfile/24)):

        ta = np.empty((144,nlat,nlon),dtype=np.float64)
        qv = np.empty((144,nlat,nlon),dtype=np.float64)
        ua = np.empty((144,nlat,nlon),dtype=np.float64)
        va = np.empty((144,nlat,nlon),dtype=np.float64)
        adv = np.empty((144,nlat,nlon),dtype=np.float64)
        mt = np.empty((144,nlat,nlon),dtype=np.float64)
        
        wrffiles = [nc.Dataset(filelist[(ff*24)]),nc.Dataset(filelist[(ff*24)+1]),nc.Dataset(filelist[(ff*24)+2])
        ,nc.Dataset(filelist[(ff*24)+3]),nc.Dataset(filelist[(ff*24)+4]),nc.Dataset(filelist[(ff*24)+5])
        ,nc.Dataset(filelist[(ff*24)+6]),nc.Dataset(filelist[(ff*24)+7]),nc.Dataset(filelist[(ff*24)+8])
        ,nc.Dataset(filelist[(ff*24)+9]),nc.Dataset(filelist[(ff*24)+10]),nc.Dataset(filelist[(ff*24)+11])
        ,nc.Dataset(filelist[(ff*24)+12]),nc.Dataset(filelist[(ff*24)+13]),nc.Dataset(filelist[(ff*24)+14])
        ,nc.Dataset(filelist[(ff*24)+15]),nc.Dataset(filelist[(ff*24)+16]),nc.Dataset(filelist[(ff*24)+17])
        ,nc.Dataset(filelist[(ff*24)+18]),nc.Dataset(filelist[(ff*24)+19]),nc.Dataset(filelist[(ff*24)+20])
        ,nc.Dataset(filelist[(ff*24)+21]),nc.Dataset(filelist[(ff*24)+22]),nc.Dataset(filelist[(ff*24)+23])]

        pres = wrf.getvar(wrffiles,"pressure",timeidx=None,method='cat')[:144,:,:,:]
        temp = wrf.getvar(wrffiles,"tk",timeidx=None,method='cat')[:144,:,:,:]
        qvap = wrf.getvar(wrffiles,"QVAPOR",timeidx=None,method='cat')[:144,:,:,:]
        uwnd = wrf.getvar(wrffiles,"ua",timeidx=None,method='cat')[:144,:,:,:]
        vwnd = wrf.getvar(wrffiles,"va",timeidx=None,method='cat')[:144,:,:,:]
        t2 = wrf.getvar(wrffiles,"T2",timeidx=None,method='cat')[:144,:,:] - 273.15
         
        # Extract variables at desired level
        ta = wrf.to_np(wrf.interplevel(temp,pres,850.))
        qv = wrf.to_np(wrf.interplevel(qvap,pres,850.))
        ua = wrf.to_np(wrf.interplevel(uwnd,pres,850.))
        va = wrf.to_np(wrf.interplevel(vwnd,pres,850.))
        
        del pres,temp,qvap,uwnd,vwnd,wwnd
        
        # Calculate the advection
        for tind in range(144):
            adv[tind,:,:] = mpcalc.advection(ta[tind,:,:], [ua[tind,:,:], va[tind,:,:]],(dx, dy), dim_order='yx')
            mt[tind,:,:] = mpcalc.advection(qv[tind,:,:], [ua[tind,:,:], va[tind,:,:]],(dx, dy), dim_order='yx')
            
        for a in range(24):
            wrffiles[a].close()
        
        # Create 1 file per day
        dataset = nc.Dataset('/g/data/w97/azh561/WRF/processed/wrfout_850hPa_%s_%s_%s.nc' %(filelist[ff*24][61:71],ensmem[mm],domain[mm]),'w') # open file

        # Create dimensions
        time = dataset.createDimension('time',None)
        lat = dataset.createDimension('lat',nlat)
        lon = dataset.createDimension('lon',nlon)

        # Create coordinate variables
        times = dataset.createVariable('time',ftimes.dtype,('time',))
        latitude = dataset.createVariable('lat',np.float64,('lat','lon',))
        longitude = dataset.createVariable('lon',np.float64,('lat','lon',))

        # Create variables
        ADV850 = dataset.createVariable('ADV850', np.float64,('time','lat','lon'))
        MT850 = dataset.createVariable('MT850', np.float64,('time','lat','lon'))
        T2M = dataset.createVariable('T2M', np.float64,('time','lat','lon'))

        # Write data
        ADV850[:] = adv[:] 
        MT850[:] = mt[:] 
        T2M[:] = t2[:] 
        times[:] = ftimes[144*ff:144*(ff+1)]
        latitude[:] = lat2d
        longitude[:] = lon2d

        # Write the file
        dataset.close()

        del ta,qv,ua,va,adv,mt,t2    
        
    del filelist,nfile
    


### Read in the extracted data and plot
- calculate the ensemble average on Gadi using ncea

In [ ]:
# Get urban
file = nc.Dataset('/g/data/w97/azh561/WRF/processed/wrfout_850hPa_%s.nc' %('d02'),'r')
ADV = file.variables['ADV'][:,:,:]
MT = file.variables['MT'][:,:,:]
file.close()

# Extract particular timesteps where the Blue Mountains advection or sea breeze 
# is active leading in/out of a heatwave